In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage

class AgentState(TypedDict):
    messages: list[Annotated[AnyMessage, add_messages]]

In [ ]:
def generate(state: AgentState) -> AgentState:
    """
    'generate' Node
    : 사용자의 질문을 받아, 응답을 생성한다.
    """
    
    messages = state['messages']
    ai_message = llm.invoke(messages)
    
    return {'messages': [ai_message]}

In [ ]:
from langgraph.graph import StateGraph, START, END

graph_builder = StateGraph(AgentState)

# nodes
graph_builder.add_node('generate', generate)

# edges
graph_builder.add_edge(START, 'generate')
graph_builder.add_edge('generate', END)

In [ ]:
graph = graph_builder.compile()

In [ ]:
from IPython.display import display, Image

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
from langchain_core.messages import HumanMessage

query = 'LangGraph를 간단히 설명해주세요.'
initial_state = {'messages': [HumanMessage(query)]}

graph.invoke(initial_state)